In [37]:
import argparse
import datetime
import os
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader as DataLoader
import shutil
import nbimporter
import basic_model as net
from audioDataset import AudioDataset

In [38]:
#set run choices
loss_plot = False
verbose = True
epoch_save = False

# set variables
train_dir = './data/training/clips/'
#test_dir = './data/testing/clips/'
gamma = 0.1
num_epochs = 10
batch_size = 32
learning_rate = 1e-4
weight_decay = 1e-5

In [44]:
device = torch.device("cpu")

#need to change based on model name
#this calls the constructor of model class
model = net.GenreClassificationANN();

#dataloader
train_dataset = AudioDataset(train_dir)
data_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

#model optimizer
model_optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay )

#model scheduler
model_scheduler = torch.optim.lr_scheduler.StepLR(model_optimizer, step_size=10, gamma=gamma)

In [47]:
def train():
    model.train()
    model.to(device)

    epoch_losses = []
    n_images = 800;
    n_batches = int(n_images/batch_size)

    for epoch in range(1, num_epochs+1):
        batch_losses = []
        print(f'Epoch #{epoch}, Start Time: {datetime.datetime.now()}')

        for audios, labels in data_loader:
            audios = audios.to(device)
            labels = labels.to(device)
        
            #calculate losses and call call model
            output = model(audios)

            ####batch loss math######
            loss = 0
            batch_losses += [float(output.data.item())]

            model_optimizer.zero_grad()
            loss.backward()
            model_optimizer.step()

        ###epoch loss math#####

        epoch_losses += [sum(batch_losses/n_images)]

        model_scheduler.step()

        if verbose:
            print(f'Loss {epoch_losses[epoch-1]}')

        if epoch_save:
            model_folder_dir = './temp_models'
            if not os.path.isdir( model_folder_dir):
                os.mkdir(model_folder_dir)

             # save temp model
            try:
                temp_model_path = model_folder_dir + '/' + str(datetime.datetime.now()) + '_epoch' + str(epoch) + '.pth'
                torch.save(model.forward.state_dict(), temp_model_path)
                if verbose:
                    print(f'Saved model for epoch {epoch} @{temp_model_path}')
            except:
                print('Epoch model save failed')

    #save final model parameters   
    torch.save(model.forward.state_dict())

    ###generate loss plot####


In [48]:
if __name__ == "__main__":
    train()

Epoch #1, Start Time: 2023-11-22 14:23:05.694741


RuntimeError: Couldn't find appropriate backend to handle uri ./data/training/clips/disco.00029.wav and format None.